In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Read the data
data = pd.read_csv('/kaggle/input/datalast/train_test_data.csv')

# Extract 'Category of transaction' and 'Type of transaction'
split_columns = data['category_transaction'].str.split('-', n=1, expand=True)
data['Category of transaction'] = split_columns[0]
data['Type of transaction'] = split_columns[1]
data.drop('category_transaction', axis=1, inplace=True)

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d %H:%M:%S')

# Create time series features
def create_features(data):
    data = data.copy()
    data['dayofweek'] = data['Date'].dt.dayofweek
    data['quarter'] = data['Date'].dt.quarter
    data['month'] = data['Date'].dt.month
    data['year'] = data['Date'].dt.year
    data['dayofyear'] = data['Date'].dt.dayofyear
    data['dayofmonth'] = data['Date'].dt.day
    data['weekofyear'] = data['Date'].dt.isocalendar().week
    return data

data = create_features(data)

# Encode categorical columns
le = LabelEncoder()
data['Category_encoded'] = le.fit_transform(data['Category of transaction'])
data['Type_encoded'] = le.fit_transform(data['Type of transaction'])
data['Id_compte_encoded'] = le.fit_transform(data['Id_compte'])

# Drop unnecessary columns
columns_to_drop = ['Id_compte', 'Type of transaction', 'Category of transaction', 'Date']
data = data.drop(columns=columns_to_drop)

# Split the data
X = data.drop(['amount_transaction'], axis=1)
Y = data['amount_transaction']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hyperparameter Tuning for XGBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

xgb_model = XGBRegressor()
grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, Y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test_scaled)

# Calculate Mean Squared Error
mse = mean_squared_error(Y_test, y_pred)
print("Mean Squared Error on Test Set:", mse)

In [ ]:
prediction_dates = pd.date_range(start='2022-11-08', end='2022-12-31')
prediction_data = pd.DataFrame({
    'Date': np.repeat(prediction_dates, len(le.classes_)),
    'Id_compte_encoded': np.tile(le.transform(le.classes_), len(prediction_dates)),
    'Category_encoded': np.tile(le.transform(le.classes_), len(prediction_dates)),
    'Type_encoded': np.tile(le.transform(le.classes_), len(prediction_dates))
})

# Extracting month and day for prediction
prediction_data = create_features(prediction_data)

# Prepare features for prediction
X_prediction = prediction_data[['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'Category_encoded', 'Type_encoded', 'Id_compte_encoded']]

# Make predictions for the evaluation period
predicted_amounts = best_model.predict(X_prediction)
prediction_data['Predicted_Amount'] = predicted_amounts
prediction_data['ID'] = prediction_data['Date'].dt.strftime('%Y-%m-%d') + '_' + le.inverse_transform(prediction_data['Id_compte_encoded'])

# Preparing the final output
final_output = prediction_data[['ID', 'Predicted_Amount']]

# Save the final output data to a CSV file
final_output.to_csv('final_output.csv', index=False)